In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType,MapType, StringType,ArrayType, FloatType, TimestampType, IntegerType
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
ModuleMetadata
from scipy import signal
from scipy.interpolate import interp1d
from cerebralcortex import Kernel
from scipy.stats import skew,kurtosis,mode
CC = Kernel("/home/jupyter/cc3_conf/", study_name='mperf')

In [2]:
stream_name = 'org.md2k.feature.motionsensehrv.decoded.rightwrist.all.activity'

In [4]:
data = CC.get_stream(stream_name,user_id='35daf881-da7c-4779-a8e9-20a3985094e2')

In [5]:
dadata.select('localtime','timestamp','day','prediction','user','version')

root
 |-- localtime: timestamp (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- start: timestamp (nullable = true)
 |-- end: timestamp (nullable = true)
 |-- day: string (nullable = true)
 |-- magnitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- prediction: string (nullable = true)
 |-- prediction_wisdm: string (nullable = true)
 |-- mean: double (nullable = true)
 |-- std: double (nullable = true)
 |-- skew: double (nullable = true)
 |-- kurtosis: double (nullable = true)
 |-- version: integer (nullable = false)
 |-- user: string (nullable = false)

